In [100]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

---

# Data Prepare

In [101]:
# Set up file path
AsIsPath = r'\\neptune\RAD\2 RuleSets\WIP\2021_Jul_RS_WIP\Score Cut 209 9000S FL_ALI\Exports\SCORECARD_06-30-21_540_AsIs_Base_460_YZH.csv'
ToBePath = r'\\neptune\RAD\2 RuleSets\WIP\2021_Jul_RS_WIP\Score Cut 209 9000S FL_ALI\Exports\SCORECARD_06-30-21_541_ToBe_AllowFL9000s_CarTier_YZH.csv'

In [102]:
# Prepare data for As-is. Put in variables that that we need for the analysis. 
asis = pd.read_csv(AsIsPath)
asis = asis[['DL_MASTER','BY1_SCORE','BY1_SEG','B1_FACTORTRUST_HIT','CONTRACT_OPENDATE','STA','DEL_STA','%_BOOK','P',"Buyer_1's_State",'BYR1_FICO','BAD_RATE1','VEH_BBCATEGORY','S_DECISION_B1','S_LTVg_B1','S_DISCg_B1']]
asis.rename(columns={"S_DECISION_B1": "AsIsDecision", "S_LTVg_B1": "AsIsATB", "S_DISCg_B1":"AsIsDisc"},inplace=True)
asis['SCOREX'] = asis['BY1_SCORE']/10
asis['SCOREX'] = asis['SCOREX'].apply(np.floor).fillna(0).astype(int)
asis.head()

,DL_MASTER,BY1_SCORE,BY1_SEG,B1_FACTORTRUST_HIT,CONTRACT_OPENDATE,STA,DEL_STA,%_BOOK,P,Buyer_1's_State,BYR1_FICO,BAD_RATE1,VEH_BBCATEGORY,AsIsDecision,AsIsATB,AsIsDisc,SCOREX
0,2954285,209.0,6J,0.0,06-01-21,D,PRE,56.74,I,NV,9002.0,0.3347,MID-SIZE CROSSOVER/SUV,D,0.5966,0.0700,20
1,2956130,225.0,6K,0.0,06-01-21,A,DEL,91.82,I,FL,NaN,0.2696,COMPACT CROSSOVER/SUV,A,1.0149,0.0400,22
2,2961030,237.0,6D,0.0,06-01-21,A,PRE,101.63,I,CA,603.0,0.2815,SPORTY CAR,A,1.1419,0.0200,23
3,2965230,244.0,6G,0.0,06-01-21,A,DEL,108.68,I,CO,608.0,0.2405,MID-SIZE CROSSOVER/SUV,A,1.1049,0.0175,24
4,2966005,238.0,6C,1.0,06-01-21,A,DEL,109.07,I,FL,672.0,0.2895,SPORTY CAR,A,1.0866,0.0360,23


In [103]:
# Check count for As-is data
nasis = asis.shape[0]
asis.shape

(16237, 17)

In [104]:
# Prepare data for To-be
tobe = pd.read_csv(ToBePath)
tobe = tobe[['DL_MASTER','S_DECISION_B1','S_LTVg_B1','S_DISCg_B1']]
tobe.rename(columns={"S_DECISION_B1": "ToBeDecision", "S_LTVg_B1": "ToBeATB", "S_DISCg_B1":"ToBeDisc"},inplace=True)
tobe.head()

,DL_MASTER,AdjLTV,ToBeDecision,ToBeATB,ToBeDisc
0,2954285,0.6745,D,0.5966,0.0700
1,2956130,1.0970,A,1.0149,0.0400
2,2961030,1.2108,A,1.1419,0.0200
3,2965230,1.1572,A,1.1049,0.0175
4,2966005,1.1642,A,1.0866,0.0360


In [105]:
# Check count for To-be data
ntobe = tobe.shape[0]
tobe.shape

(16235, 5)

In [106]:
# Join As-is and To-be
df = pd.merge(asis, tobe, on='DL_MASTER')
df.head()

,DL_MASTER,BY1_SCORE,BY1_SEG,B1_FACTORTRUST_HIT,CONTRACT_OPENDATE,STA,DEL_STA,%_BOOK,P,Buyer_1's_State,BYR1_FICO,BAD_RATE1,VEH_BBCATEGORY,AsIsDecision,AsIsATB,AsIsDisc,SCOREX,AdjLTV,ToBeDecision,ToBeATB,ToBeDisc
0,2954285,209.0,6J,0.0,06-01-21,D,PRE,56.74,I,NV,9002.0,0.3347,MID-SIZE CROSSOVER/SUV,D,0.5966,0.0700,20,0.6745,D,0.5966,0.0700
1,2956130,225.0,6K,0.0,06-01-21,A,DEL,91.82,I,FL,NaN,0.2696,COMPACT CROSSOVER/SUV,A,1.0149,0.0400,22,1.0970,A,1.0149,0.0400
2,2961030,237.0,6D,0.0,06-01-21,A,PRE,101.63,I,CA,603.0,0.2815,SPORTY CAR,A,1.1419,0.0200,23,1.2108,A,1.1419,0.0200
3,2965230,244.0,6G,0.0,06-01-21,A,DEL,108.68,I,CO,608.0,0.2405,MID-SIZE CROSSOVER/SUV,A,1.1049,0.0175,24,1.1572,A,1.1049,0.0175
4,2966005,238.0,6C,1.0,06-01-21,A,DEL,109.07,I,FL,672.0,0.2895,SPORTY CAR,A,1.0866,0.0360,23,1.1642,A,1.0866,0.0360


In [107]:
# Check count for joined data
njoin = df.shape[0]
df.shape

(16235, 21)

In [108]:
# Get Date Range
MinDate = min(df['CONTRACT_OPENDATE'])
MaxDate = max(df['CONTRACT_OPENDATE'])
DateRange = 'Deals opened from ' + str(MinDate) + ' to ' + str(MaxDate)

DateRange

'Deals opened from 06-01-21 to 06-30-21'

In [109]:
# Apply filters for joined data
filters = []

filter1 = 'SCOREX is not null'
df = df[df['SCOREX'] != 0]     
filters.append(filter1)

filter2 = 'As-is decision is not null'
df = df[df['AsIsDecision'].notnull()]  
filters.append(filter2)

filter3 = 'To-be decision is not null'
df = df[df['ToBeDecision'].notnull()]    
filters.append(filter3)

df.head()

,DL_MASTER,BY1_SCORE,BY1_SEG,B1_FACTORTRUST_HIT,CONTRACT_OPENDATE,STA,DEL_STA,%_BOOK,P,Buyer_1's_State,BYR1_FICO,BAD_RATE1,VEH_BBCATEGORY,AsIsDecision,AsIsATB,AsIsDisc,SCOREX,AdjLTV,ToBeDecision,ToBeATB,ToBeDisc
0,2954285,209.0,6J,0.0,06-01-21,D,PRE,56.74,I,NV,9002.0,0.3347,MID-SIZE CROSSOVER/SUV,D,0.5966,0.0700,20,0.6745,D,0.5966,0.0700
1,2956130,225.0,6K,0.0,06-01-21,A,DEL,91.82,I,FL,NaN,0.2696,COMPACT CROSSOVER/SUV,A,1.0149,0.0400,22,1.0970,A,1.0149,0.0400
2,2961030,237.0,6D,0.0,06-01-21,A,PRE,101.63,I,CA,603.0,0.2815,SPORTY CAR,A,1.1419,0.0200,23,1.2108,A,1.1419,0.0200
3,2965230,244.0,6G,0.0,06-01-21,A,DEL,108.68,I,CO,608.0,0.2405,MID-SIZE CROSSOVER/SUV,A,1.1049,0.0175,24,1.1572,A,1.1049,0.0175
4,2966005,238.0,6C,1.0,06-01-21,A,DEL,109.07,I,FL,672.0,0.2895,SPORTY CAR,A,1.0866,0.0360,23,1.1642,A,1.0866,0.0360


In [110]:
# Check count for filtered joined data
nfilter = df.shape[0]
df.shape

(15289, 21)

In [111]:
# Write down the filters we used to clean up the joined data
FilterUsed = 'Filters: '
for f in filters: 
    if filters.index(f) != len(filters)-1:
        FilterUsed = FilterUsed + f + ', '
    else: 
        FilterUsed = FilterUsed + f + '.'

FilterUsed

'Filters: SCOREX is not null, As-is decision is not null, To-be decision is not null.'

In [112]:
# Track the count of data cleaning process
AsIsCount = 'As-is Count: ' + str(nasis) 
ToBeCount = 'To-be Count: ' + str(ntobe) 
JoinedCount = 'Joined Count: ' + str(njoin) 
FilteredCount = 'Filtered Count: ' + str(nfilter)

print(AsIsCount)
print(ToBeCount)
print(JoinedCount)
print(FilteredCount)

As-is Count: 16237
To-be Count: 16235
Joined Count: 16235
Filtered Count: 15289


---

# Analysis

In [113]:
# Set up Subpopulation Group (Pick one of the 2 options below. Comment out the one you don't use.)

#    Option 1. Put in the variable name if the subpopulation is splitted by an existing variable in the dataset
GroupName = "Buyer_1's_State"

#    Option 2. Define a new variable and split subpopulation base on the new variable
#def sub_pop(fico):
#    if pd.isna(fico):
#        return 'Missing'
#    elif fico >= 9001 and fico <=9003:
#        return '9000s'
#    elif fico >=250 and fico <=900:
#        return 'Valid'
#    else: return 'Other'
#df['FICO_Seg'] = df['BYR1_FICO'].map(sub_pop)
#GroupName = "FICO_Seg"

In [114]:
# Initiate a list of dataframes to put in Analysis Tab
dfs = []

In [115]:
# Function of table formatting

# Format tables with counts
def FormatCnt(df):
    df = df.fillna(0).astype(int)
    df.columns = df.columns.droplevel()
    df.rename(columns={"All": "Total"},index = {"All": "Total"} ,inplace=True)
    df.insert(0, 'Total', df.pop("Total"))
    return df

# Format tables with values
def FormatVal(df):
    df = df.fillna(0)
    df = df.round(1)
    df.columns = df.columns.droplevel()
    df.rename(columns={"All": "Total"},index = {"All": "Total"} ,inplace=True)
    df.insert(0, 'Total', df.pop("Total"))
    df.replace(0.0, np.nan, inplace=True)
    return df

In [116]:
# Decision Change Count
DecisionChange = pd.pivot_table(df, values=['DL_MASTER'], index=['AsIsDecision'], columns=['ToBeDecision'], aggfunc=len,margins=True)
DecisionChange = FormatCnt(DecisionChange)
DecisionChange.name = 'Decision Change'
dfs.append(DecisionChange)
DecisionChange

ToBeDecision,Total,A,C,D
AsIsDecision,,,,
A,8610,8606,4,0
C,1425,0,1425,0
D,5254,60,41,5153
Total,15289,8666,1470,5153


In [117]:
# Prep data for Approved to Approved population
dfapp = df[(df.AsIsDecision=='A') & (df.ToBeDecision=='A')]
dfapp.shape

(8606, 22)

In [118]:
# Approved to Approved Count
ApprovedToApprovedCount = pd.pivot_table(dfapp, values=['DL_MASTER'], index=[GroupName], columns=['SCOREX'], aggfunc=len,margins=True)
ApprovedToApprovedCount = FormatCnt(ApprovedToApprovedCount)
ApprovedToApprovedCount.name = 'Approved to Approved Count'
dfs.append(ApprovedToApprovedCount)
ApprovedToApprovedCount

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
FICO_Seg,,,,,,,,,,,,,
9000s,1306,0,1,0,470,372,283,133,33,12,2,0,0
Missing,3247,0,0,0,589,2038,402,149,62,7,0,0,0
Valid,4053,1,0,369,706,711,694,566,492,299,149,44,22
Total,8606,1,1,369,1765,3121,1379,848,587,318,151,44,22


In [119]:
# As-is ATB
AsIsAtb = pd.pivot_table(dfapp, values=['AsIsATB'], index=[GroupName], columns=['SCOREX'], aggfunc=np.mean,margins=True)
AsIsAtb = AsIsAtb*100
AsIsAtb = FormatVal(AsIsAtb)
AsIsAtb.name = 'As Is ATB'
dfs.append(AsIsAtb)
AsIsAtb

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
FICO_Seg,,,,,,,,,,,,,
9000s,97.7,NaN,105.8,NaN,92.8,98.1,101.2,104.7,101.6,107.0,90.1,NaN,NaN
Missing,96.0,NaN,NaN,NaN,91.5,96.6,98.9,99.1,96.6,86.5,NaN,NaN,NaN
Valid,98.0,98.6,NaN,92.8,92.1,95.2,100.5,102.2,103.1,101.7,100.5,97.0,93.3
Total,97.2,98.6,105.8,92.8,92.1,96.4,100.2,102.1,102.3,101.6,100.4,97.0,93.3


In [120]:
# To-be ATB
ToBeAtb = pd.pivot_table(dfapp, values=['ToBeATB'], index=[GroupName], columns=['SCOREX'], aggfunc=np.mean,margins=True)
ToBeAtb = ToBeAtb*100
ToBeAtb = FormatVal(ToBeAtb)
ToBeAtb.name = 'To Be ATB'
dfs.append(ToBeAtb)
ToBeAtb

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
FICO_Seg,,,,,,,,,,,,,
9000s,97.7,NaN,105.8,NaN,92.8,98.1,101.2,104.7,101.6,107.0,90.1,NaN,NaN
Missing,96.0,NaN,NaN,NaN,91.5,96.6,98.9,99.1,96.6,86.5,NaN,NaN,NaN
Valid,98.0,98.6,NaN,92.8,92.1,95.2,100.5,102.2,103.1,101.7,100.5,97.0,93.3
Total,97.2,98.6,105.8,92.8,92.1,96.4,100.2,102.1,102.3,101.6,100.4,97.0,93.3


In [121]:
# As-is Disc
AsIsDisc = pd.pivot_table(dfapp, values=['AsIsDisc'], index=[GroupName], columns=['SCOREX'], aggfunc=np.mean,margins=True)
AsIsDisc = AsIsDisc*100
AsIsDisc = FormatVal(AsIsDisc)
AsIsDisc.name = 'As Is Disc'
dfs.append(AsIsDisc)
AsIsDisc

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
FICO_Seg,,,,,,,,,,,,,
9000s,5.0,NaN,6.5,NaN,7.3,4.8,3.1,2.7,1.5,2.5,1.0,NaN,NaN
Missing,4.7,NaN,NaN,NaN,7.5,4.5,2.9,2.3,1.8,1.4,NaN,NaN,NaN
Valid,4.7,6.0,NaN,7.5,7.2,5.1,4.0,3.7,2.8,2.5,2.5,2.1,1.7
Total,4.7,6.0,6.5,7.5,7.3,4.7,3.5,3.3,2.6,2.5,2.5,2.1,1.7


In [122]:
# To-be Disc
ToBeDisc = pd.pivot_table(dfapp, values=['ToBeDisc'], index=[GroupName], columns=['SCOREX'], aggfunc=np.mean,margins=True)
ToBeDisc = ToBeDisc*100
ToBeDisc = FormatVal(ToBeDisc)
ToBeDisc.name = 'To Be Disc'
dfs.append(ToBeDisc)
ToBeDisc

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
FICO_Seg,,,,,,,,,,,,,
9000s,5.0,NaN,6.5,NaN,7.3,4.8,3.1,2.7,1.5,2.5,1.0,NaN,NaN
Missing,4.7,NaN,NaN,NaN,7.5,4.5,2.9,2.3,1.8,1.4,NaN,NaN,NaN
Valid,4.7,6.0,NaN,7.5,7.2,5.1,4.0,3.7,2.8,2.5,2.5,2.1,1.7
Total,4.7,6.0,6.5,7.5,7.3,4.7,3.5,3.3,2.6,2.5,2.5,2.1,1.7


In [123]:
# ATB Diff
AtbDiff = ToBeAtb.fillna(0)-AsIsAtb.fillna(0)
AtbDiff.name = 'ATB Diff'
dfs.append(AtbDiff)
AtbDiff

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
FICO_Seg,,,,,,,,,,,,,
9000s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Missing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Valid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Total,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
# Disc Diff
DiscDiff = ToBeDisc.fillna(0)-AsIsDisc.fillna(0)
DiscDiff.name = 'Disc Diff'
dfs.append(DiscDiff)
DiscDiff

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
FICO_Seg,,,,,,,,,,,,,
9000s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Missing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Valid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Total,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

# Approval Volume Change

In [125]:
# Prep As-is Approved and To-be Approved data
dfasisapp = df[df.AsIsDecision=='A']
dftobeapp = df[df.ToBeDecision=='A']

# Calculate the count of As-is Approved and To-be Approved population
N_AsIsA = len(dfasisapp)
N_ToBeA = len(dftobeapp)

# Initiate a list to save tables if there's any approval volume change 
VolChgLst = []

# Check if approval volume changes. If yes, append the analysis tables to the list created above
if N_AsIsA != N_ToBeA:
    # As-is Approved Count
    AsIsApprovedCount = pd.pivot_table(dfasisapp, values=['DL_MASTER'], index=[GroupName], columns=['SCOREX'], aggfunc=len,margins=True)
    AsIsApprovedCount = FormatCnt(AsIsApprovedCount)
    AsIsApprovedCount.name = 'As Is Approved Count'
    VolChgLst.append(AsIsApprovedCount)
    
    # To-be Approved Count
    ToBeApprovedCount = pd.pivot_table(dftobeapp, values=['DL_MASTER'], index=[GroupName], columns=['SCOREX'], aggfunc=len,margins=True)
    ToBeApprovedCount = FormatCnt(ToBeApprovedCount)
    ToBeApprovedCount.name = 'To Be Approved Count'
    VolChgLst.append(ToBeApprovedCount)
    
    # Volume Change Count
    VolumeChange = ToBeApprovedCount - AsIsApprovedCount
    VolumeChange.name = 'Volume Change Count'
    VolChgLst.append(VolumeChange)
    
    # Volume Change in %
    VolumeChangePct = 100*(ToBeApprovedCount - AsIsApprovedCount)/AsIsApprovedCount
    VolumeChangePct = VolumeChangePct.round(1)
    VolumeChangePct.name = 'Volume Change in %'
    VolChgLst.append(VolumeChangePct)

dfs.extend(VolChgLst)

In [126]:
#AsIsApprovedCount

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
FICO_Seg,,,,,,,,,,,,,
9000s,1306,0,1,0,470,372,283,133,33,12,2,0,0
Missing,3247,0,0,0,589,2038,402,149,62,7,0,0,0
Valid,4057,1,0,369,707,713,695,566,492,299,149,44,22
Total,8610,1,1,369,1766,3123,1380,848,587,318,151,44,22


In [127]:
#ToBeApprovedCount

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
FICO_Seg,,,,,,,,,,,,,
9000s,1366,0,1,0,530,372,283,133,33,12,2,0,0
Missing,3247,0,0,0,589,2038,402,149,62,7,0,0,0
Valid,4053,1,0,369,706,711,694,566,492,299,149,44,22
Total,8666,1,1,369,1825,3121,1379,848,587,318,151,44,22


In [128]:
#VolumeChange

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
FICO_Seg,,,,,,,,,,,,,
9000s,60,0,0,0,60,0,0,0,0,0,0,0,0
Missing,0,0,0,0,0,0,0,0,0,0,0,0,0
Valid,-4,0,0,0,-1,-2,-1,0,0,0,0,0,0
Total,56,0,0,0,59,-2,-1,0,0,0,0,0,0


In [129]:
#VolumeChangePct

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
FICO_Seg,,,,,,,,,,,,,
9000s,4.6,NaN,0.0,NaN,12.8,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Missing,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
Valid,-0.1,0.0,NaN,0.0,-0.1,-0.3,-0.1,0.0,0.0,0.0,0.0,0.0,0.0
Total,0.7,0.0,0.0,0.0,3.3,-0.1,-0.1,0.0,0.0,0.0,0.0,0.0,0.0


---

# Decision and Pricing Changes

In [130]:
# Prepare datasets to review if there're any deals with decision changes or pricing changes.

# Initiate a list 
ChgList = []

In [131]:
# Decision Changes

# If there's any decision change, put the dataset into the list created above
for i in ['A','C','D']:
    for j in ['A','C','D']:
        if i != j:
            dfdecchg = df[(df.AsIsDecision==i) & (df.ToBeDecision==j)]
            dfdecchg.name = i + ' to ' + j
            count = len(dfdecchg)
            if count > 0:
                ChgList.append(dfdecchg)

In [132]:
# Pricing Changes

# If there's any ATB change, put the dataset into the list created above
df_atb_chg = dfapp[dfapp.AsIsATB != dfapp.ToBeATB]
df_atb_chg.name = 'ATB Change'
if len(df_atb_chg) > 0:
    ChgList.append(df_atb_chg)

# If there's any Disc change, put the dataset into the list created above
df_disc_chg = dfapp[dfapp.AsIsDisc != dfapp.ToBeDisc]
df_disc_chg.name = 'Disc Change'
if len(df_disc_chg) > 0:
    ChgList.append(df_disc_chg)

# Export the Result to Excel File

In [133]:
# Check the tables in analysis tab
for df in dfs:
    print(df.name)

Decision Change
Approved to Approved Count
As Is ATB
To Be ATB
As Is Disc
To Be Disc
ATB Diff
Disc Diff
As Is Approved Count
To Be Approved Count
Volume Change Count
Volume Change in %


In [134]:
# Check the tables in Decision Change or Pricing Change Tabs
for df in ChgList:
    print(df.name)

A to C
D to A
D to C


In [135]:
# Funtion of Excel Export
def multiple_dfs(df_list, sheets, file_name, spaces, DecisionChgList=[]):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter') 

    row = 8
    for dataframe in df_list:
        dataframe.to_excel(writer,sheet_name=sheets,startrow=row , startcol=1) 
        writer.sheets[sheets].write(row, 0, dataframe.name)
        row = row + len(dataframe.index) + spaces + 1
    writer.sheets[sheets].write(0, 0, DateRange)
    writer.sheets[sheets].write(1, 0, FilterUsed)
    writer.sheets[sheets].write(2, 0, 'Count Tracking')
    writer.sheets[sheets].write(2, 1, AsIsCount)
    writer.sheets[sheets].write(3, 1, ToBeCount)
    writer.sheets[sheets].write(4, 1, JoinedCount)
    writer.sheets[sheets].write(5, 1, FilteredCount)
    writer.sheets[sheets].set_column(0,0,30)
    writer.sheets[sheets].set_column(1,1,20)
    
    if len(DecisionChgList) > 0:
        for df in DecisionChgList:
            df.to_excel(writer, sheet_name=df.name, index=False)
    writer.save()

# Run function
multiple_dfs(dfs, 'Analysis', 'Analysis1_YZH.xlsx', 1, ChgList)